In [2]:
#!pip install tensorflow

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4835 sha256=33f964a4805e70eef342295f850a9a986e53749bf5b5526889b655b72b15fd25
  Stored in directory: c:\users\javier\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-py3-none-any.whl size=19558 sha256=78a8893d747499fbd93a0311708a79bd5910120951013ad962

Proyecto compiado de: https://towardsdatascience.com/how-to-build-your-own-chatbot-using-deep-learning-bb41f970e281

In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
#keras preprocesamiento
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

Lee el json y crea los conjuntos de datos para entrenar el modelo

In [8]:

with open('../testFiles/intentsCV.json', encoding='utf-8') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [9]:
print(len(training_sentences))
print(len(training_labels))
print(len(labels))
print(len(responses))
responses

75
75
14
14


[['Hola, qué tal estas?', 'Hola', 'Qué tal?'],
 ['Adiós', 'Pasa un buen día', 'Espero que hablemos pronto'],
 ['De nada', 'Un placer'],
 ['Me gustaría guiarte a través del CV de Javier Aparicio',
  'Puedo hablarte de la experiendcia profesional de Javier Aparicio'],
 ['Mi nombre es Bot, Javi Bot',
  'Me llamo Javi Bot',
  'Puedes dirigirte a mi como Javi Bot'],
 ['Estoy formandome como Data Scientist',
  'Estoy estudiando  Machine Leraning'],
 ['Me falta del TFG para obtener el grado de Ingeniería Informática y, actualmente, estoy haciendo un Bootcamp sobre Data Science'],
 ['Ultimamente he estado en ANCERT y anteriormente estuve 14 años en el CSIC'],
 ['En ANCERT hice labores de analista programador VBA para análisis de datos y Java para backend'],
 ['En el CSIC hice todo tipo de labores informáticas, aunque sobre todo programar, tanto en VBA y Java, para el análisis de datos científicos'],
 ['Sí, tengo bastante experiencia',
  'Poseo gran conocimiento',
  'Lo he usado en diferentes p

In [10]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)
len(training_labels)
training_labels

array([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  7,  7,  7,  7,  7,  7,
       13, 13, 13, 13, 13, 13, 13, 13,  0,  0,  0,  0,  0,  0,  9,  9,  9,
        9,  9, 10, 10, 10, 12, 12, 12,  5,  5,  5,  1,  1,  1,  4,  4,  4,
       11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,  6,  6,  6,  6,  3,
        3,  3,  3,  3,  2,  2,  2])

In [11]:
lbl_encoder.inverse_transform(training_labels)

array(['greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'greeting', 'greeting', 'greeting', 'greeting',
       'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye',
       'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks',
       'thanks', 'thanks', 'thanks', 'about', 'about', 'about', 'about',
       'about', 'about', 'name', 'name', 'name', 'name', 'name', 'now',
       'now', 'now', 'studies', 'studies', 'studies', 'experience',
       'experience', 'experience', 'ancert', 'ancert', 'ancert', 'csic',
       'csic', 'csic', 'skills', 'skills', 'skills', 'skills', 'skills',
       'skills', 'skills', 'skills', 'skills', 'skills', 'skills',
       'skills', 'future', 'future', 'future', 'future', 'createaccount',
       'createaccount', 'createaccount', 'createaccount', 'createaccount',
       'complaint', 'complaint', 'complaint'], dtype='<U13')

Tokenización de frases textuales con Tokenizer de keras

In [12]:
training_sentences

['Hi',
 'Hey',
 'Is anyone there?',
 'Hola',
 'Eo',
 'Hello',
 'Yep!',
 'Qué tal?',
 'Buenos días',
 'Buenas tardes',
 'Javi Bot',
 'Bye',
 'See you later',
 'Goodbye',
 'Adiós',
 'Hasta luego',
 'Hasta siempre',
 'Thanks',
 'Thank you',
 "That's helpful",
 'Thanks for the help',
 'Gracias',
 'Ok',
 'Perfecto',
 'Muy bien',
 'Who are you?',
 'What are you?',
 'Who you are?',
 '¿En qué puedes ayudarme?',
 '¿qué haces?',
 '¿Para qué sirves?',
 'what is your name',
 'what should I call you',
 '¿Quién eres?',
 '¿Cómo te llamas?',
 'whats your name?',
 '¿Qué haces actualmente?',
 '¿Dónde estás ahora?',
 '¿Dónde trabajas?',
 'Formación',
 'Qué has esudiado?',
 'Tienes estudios superiores',
 '¿Qué has hecho?',
 '¿Dónde has trabajado?',
 '¿qué empresas?',
 'ANCERT',
 'Agencia Notarial de Certificación',
 'último trabajo',
 'CSIC',
 'Consejo Superior de Investigación Científica',
 'trabajo anterior',
 'JAVA',
 'Python',
 'SQL',
 'PHP',
 'VBA',
 'bases de datos',
 'Mysql',
 'Spring boot',
 'JPA'

Tratamiento de los textos para alimentar la red neuronal, Sería interesante probar una lematización en la tokenización

In [13]:
vocab_size = 1000 #número de palabras del vocabulario
embedding_dim = 16
max_len = 20 #tamaño del vector que representará al documento
oov_token = "<OOV>" #ayuda a identificar palabras que quedan fuera del vocabulario

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)#crea la bolsa de palabras
tokenizer.fit_on_texts(training_sentences)#con las palabras de los documentos (oraciones)
word_index = tokenizer.word_index
#print(tokenizer.word_counts) #cuanto se repite una palabra
#print(tokenizer.document_count) #número de docs (frases)
print(tokenizer.word_index)#palabra, índice
#print(tokenizer.word_docs)#palabra índice de documento

sequences = tokenizer.texts_to_sequences(training_sentences) #transforma las frases a vectores de ínidces de palabras
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)# lleva las secuencias a vectores del mismo tamaño


{'<OOV>': 1, 'you': 2, 'a': 3, 'qué': 4, 'to': 5, 'account': 6, '¿qué': 7, 'i': 8, 'de': 9, 'is': 10, 'are': 11, 'what': 12, 'te': 13, '¿dónde': 14, 'has': 15, 'trabajo': 16, 'create': 17, 'new': 18, 'complaint': 19, 'there': 20, 'hasta': 21, 'thanks': 22, 'for': 23, 'who': 24, 'haces': 25, 'your': 26, 'name': 27, 'el': 28, 'how': 29, 'open': 30, 'want': 31, 'an': 32, 'hi': 33, 'hey': 34, 'anyone': 35, 'hola': 36, 'eo': 37, 'hello': 38, 'yep': 39, 'tal': 40, 'buenos': 41, 'días': 42, 'buenas': 43, 'tardes': 44, 'javi': 45, 'bot': 46, 'bye': 47, 'see': 48, 'later': 49, 'goodbye': 50, 'adiós': 51, 'luego': 52, 'siempre': 53, 'thank': 54, "that's": 55, 'helpful': 56, 'the': 57, 'help': 58, 'gracias': 59, 'ok': 60, 'perfecto': 61, 'muy': 62, 'bien': 63, '¿en': 64, 'puedes': 65, 'ayudarme': 66, '¿para': 67, 'sirves': 68, 'should': 69, 'call': 70, '¿quién': 71, 'eres': 72, '¿cómo': 73, 'llamas': 74, 'whats': 75, 'actualmente': 76, 'estás': 77, 'ahora': 78, 'trabajas': 79, 'formación': 80, 'e

In [14]:
sequences

[[33],
 [34],
 [10, 35, 20],
 [36],
 [37],
 [38],
 [39],
 [4, 40],
 [41, 42],
 [43, 44],
 [45, 46],
 [47],
 [48, 2, 49],
 [50],
 [51],
 [21, 52],
 [21, 53],
 [22],
 [54, 2],
 [55, 56],
 [22, 23, 57, 58],
 [59],
 [60],
 [61],
 [62, 63],
 [24, 11, 2],
 [12, 11, 2],
 [24, 2, 11],
 [64, 4, 65, 66],
 [7, 25],
 [67, 4, 68],
 [12, 10, 26, 27],
 [12, 69, 8, 70, 2],
 [71, 72],
 [73, 13, 74],
 [75, 26, 27],
 [7, 25, 76],
 [14, 77, 78],
 [14, 79],
 [80],
 [4, 15, 81],
 [82, 83, 84],
 [7, 15, 85],
 [14, 15, 86],
 [7, 87],
 [88],
 [89, 90, 9, 91],
 [92, 16],
 [93],
 [94, 95, 9, 96, 97],
 [16, 98],
 [99],
 [100],
 [101],
 [102],
 [103],
 [104, 9, 105],
 [106],
 [107, 108],
 [109],
 [110],
 [111, 9, 112],
 [113],
 [4, 13, 114, 115, 116, 28, 117],
 [118, 119, 120, 28, 121],
 [16, 122],
 [13, 123, 124, 125, 126],
 [8, 127, 5, 17, 3, 18, 6],
 [29, 5, 30, 3, 18, 6],
 [8, 31, 5, 17, 32, 6],
 [128, 2, 17, 32, 6, 23, 129],
 [29, 5, 30, 3, 18, 6],
 [130, 3, 19],
 [8, 31, 5, 131, 3, 19],
 [20, 10, 3, 19, 132,

Creación de la red neuronal: Secuentials

In [15]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 16)            16000     
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 14)                238       
Total params: 16,782
Trainable params: 16,782
Non-trainable params: 0
_________________________________________________________________


In [16]:
epochs = 500 #número de veces en que todos los datos de entrenamiento pasan por la red en el proceso. Tip: aumentar el número de epoch hasta que la accuracy de los datos de validación (no los de entrenamiento) empiece a decrecer
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 299/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6091 - accuracy: 0.8508
Epoch 300/500
3/3 [==============================] - 0s 8ms/step - loss: 0.6026 - accuracy: 0.8129
Epoch 301/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6092 - accuracy: 0.8168
Epoch 302/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6105 - accuracy: 0.8051
Epoch 303/500
3/3 [==============================] - 0s 4ms/step - loss: 0.5966 - accuracy: 0.8547
Epoch 304/500
3/3 [==============================] - 0s 8ms/step - loss: 0.6218 - accuracy: 0.8274
Epoch 305/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6182 - accuracy: 0.8118
Epoch 306/500
3/3 [==============================] - 0s 8ms/step - loss: 0.5973 - accuracy: 0.8430
Epoch 307/500
3/3 [==============================] - 0s 4ms/step - loss: 0.6130 - accuracy: 0.8012
Epoch 308/500
3/3 [==============================] - 0s 4ms/step - loss: 0.5552 - accuracy: 0.8442
Epoch 309/

Salvo el modelo

In [18]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


In [3]:
model = keras.models.load_model('chat_model')

In [6]:
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

    # load label encoder object
with open('label_encoder.pickle', 'rb') as enc:
    lbl_encoder = pickle.load(enc)

In [10]:
import json
with open('../testFiles/intentsCV.json', encoding='utf-8') as file:
    data = json.load(file)

In [45]:
import random
result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(["gracias"]),
                                             truncating='post', maxlen=20))
print([np.argmax(result)])
tag = lbl_encoder.inverse_transform([np.argmax(result)])

print(result)
print(tag)

for i in data['intents']:
    if i['tag'] == tag:
        print("ChatBot:", np.random.choice(i['responses']))

[13]
[[3.1820178e-02 3.8853264e-11 2.4975441e-05 2.5423130e-06 3.7118504e-04
  1.4700639e-15 3.2044053e-15 3.7280038e-02 3.9225297e-06 8.1928559e-15
  6.2934008e-10 1.8491271e-03 2.5047932e-03 9.2614323e-01]]
['thanks']
ChatBot: Un placer
